In [4]:
import numpy as np
from scipy import stats

In [5]:
a = 10
b = 5

print(np.log(a-b))
print(np.log(a/b))

1.6094379124341003
0.6931471805599453


### Forward algorithm explained
See p. 48 in Zucchini

For $t=0$:

- $\alpha_{0} = \delta \cdot P(x_0)$
- $w_0 = \sum_{j=1}^m \alpha_0(j) =  \alpha_0 \cdot \textbf{1'} $
- $$\phi_0 = \frac{\alpha_0}{w_0}  $$
- $$log L_0 = log(w_0)  $$
- $$ log (\alpha_0) = logL_0 + log(\phi_0) $$

For $t\in 1..T$.
- $\alpha_t = \phi_{t-1} \cdot \Gamma \cdot P(x_t)$
- $w_t = \sum_{j=1}^m \alpha_t(j) =  \alpha_t \cdot \textbf{1'} $
- $$\phi_t = \frac{\alpha_t}{w_t}  $$
- $$log L_t = L_{t-1} +  log(w_t)  $$
- $$ log (\alpha_t) = logL_t + log(\phi_t) $$

From this it is clear that:

$logL_T = \sum_{t=1}^T log(w_t) = \sum_{t=1}^T log(\alpha_t\cdot \textbf{1}^T) =
 \sum_{t=1}^T log(\phi_{t-1} \cdot \Gamma \cdot P(x_t)\cdot \textbf{1}^T) $

 Thus the log likelihood follows directly from p. 48 in Zucchini.

 TODO $\alpha_T$ however doesn't seem to match as:

$log(\alpha_T) = \sum_{t=0}^T [log(w_t)] + log(\frac{\alpha_T}{w_T}) $

Since $log(x)+log(y)=log(x*y)$ then:

$log(\alpha_T) = log(w_0*w_1*...*w_{t-1}*w_t*\frac{\alpha_T}{w_T}) $

$log(\alpha_T) = log(w_0*w_1*...*w_{t-1}*\alpha_T) $

$\alpha_T = \prod_{t=0}^T (\alpha_t\cdot \textbf{1}) * \alpha_T  $

WHICH DOESn't MATCH P. 48 !!

### Rewriting $f_{jk} = \hat v_{jk}(t) $
To make the code more efficient

From the definition on p. 71 we have

$f_{jk} = \sum_{t=2}^T \hat v_{jk}(t)$

$ \hat v_{jk}(t) = \gamma_{jk}\alpha_{t-1}(j)p_k(x_t)\beta_t(k)/L_T $

Since $\gamma_{jk}$ doesn't depend on t we can move it outside expression in $f_{jk}$

$f_{jk} = \gamma_{jk} \sum_{t=2}^T \alpha_{t-1}(j)p_k(x_t)\beta_t(k)/L_T $

We can then denote $\alpha_j$, $\beta_k$ and $P_k(x)$ as 1 x T vectors containing all time-dependent values given the state j and k to obtain:
$f_{jk} = \gamma_{jk} \sum_{jk} \alpha_{j}P_k(x)\beta_k/L_T $

Which now only only requires looping through possible values of j and k.

In [19]:
from scipy import stats

obs = [0.1, 0.15, 0.2, 1]
print(stats.norm.pdf(0, loc=0, scale=0.5, ))
print(stats.norm.pdf(0, loc=1, scale=3, ))
print(stats.norm.pdf(0, loc=[0,1], scale=[0.5,3], ))

0.7978845608028654
0.12579440923099774
[0.79788456 0.12579441]


In [7]:
a = np.array([[1,2],[3,4]])
b = np.array([2,4]).reshape((2,1))
c = 2
print(a)
print(b)
#print(b)

a /b

[[1 2]
 [3 4]]
[[2]
 [4]]


array([[0.5 , 1.  ],
       [0.75, 1.  ]])

In [8]:
b[-1]

array([4])

In [9]:
a[-1,1]

4

In [15]:
b = np.array([0.2,0.8])
A = np.array([[0.5,0],[0,0.5]])
T = np.array([[5,6],[2,1]])

T @A @ b

array([2.9, 0.6])

In [16]:
A = np.array([0.5,0.5])

T @ (A*b)

array([2.9, 0.6])

In [25]:
np.full((2,2), 5)

array([[5, 5],
       [5, 5]])

In [29]:
def _log_multivariate_normal_density_diag(X, means, covars):
    """Compute Gaussian log-density at X for a diagonal model."""
    # X: (ns, nf); means: (nc, nf); covars: (nc, nf) -> (ns, nc)
    n_samples, n_dim = X.shape
    # Avoid 0 log 0 = nan in degenerate covariance case.
    covars = np.maximum(covars, np.finfo(float).tiny)
    with np.errstate(over="ignore"):
        return -0.5 * (n_dim * np.log(2 * np.pi)
                       + np.log(covars).sum(axis=-1)
                       + ((X[:, None, :] - means) ** 2 / covars).sum(axis=-1))

_log_multivariate_normal_density_diag([0,1,2,3,4], 2, 3)

AttributeError: 'list' object has no attribute 'shape'